In [2]:
import torch
import numpy as np
import scipy.io as sio
import os

In [3]:
if torch.cuda.is_available():
   device = torch.device("cuda")
   print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [43]:
# Paramètres du modèle
N = 2 # dim de l'input x
m = 10 # dim de l'output y
T = 1 # borne sup pour x
nb_points = 10 
nb_maps = 10 # nombre d'images que l'on va générer
nb_blocks = 2 # nombre de blocs dans le modèle
M = 10 # nombre d'échantillons pour l'approximation de Monte Carlo

In [5]:
omega = torch.tensor([[0, 1] for _ in range(N)])
x = torch.rand(nb_points, N) * (omega[:, 1] - omega[:, 0]) + omega[:, 0] # all the coordinates of x are between 0 and 1
print(f'La dimension de x est {x.size()}')

La dimension de x est torch.Size([10, 2])


In [6]:
class Block(torch.nn.Module):
    def __init__(self, m) -> None:
        super(Block, self).__init__()
        self.fc = torch.nn.Linear(m, m)
        self.activation = torch.nn.Tanh()

    def forward(self, x):
        identity = x
        x = self.fc(x)
        x = self.activation(x)
        x = self.fc(x)
        x = self.activation(x)
        return x + identity
    
class Model(torch.nn.Module):
    def __init__(self, N, m, nb_blocks) -> None:
        super(Model, self).__init__()

    def forward(self, x):
        x = torch.nn.Linear(N, m)(x)
        for _ in range(nb_blocks):
            x = Block(10)(x)
        return x

In [38]:
def conv(x : torch.Tensor, dx = True, delta : float = 1/nb_points) -> torch.Tensor:
    if dx:
        convx = torch.zeros(x.shape)
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                if j == 0 :
                    convx[i, j] = (x[i, j+1] - x[i, j])/delta # forward 
                elif j == x.shape[1] - 1:
                    convx[i, j] = (x[i, j] - x[i, j-1])/delta # backward
                else:
                    convx[i, j] = (x[i, j+1] - x[i, j-1])/(2*delta) # central
        return convx
    else:
        convy = torch.zeros(x.shape)
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                if i == 0 :
                    convy[i, j] = (x[i+1, j] - x[i, j])/delta # forward
                elif i == x.shape[0] - 1:
                    convy[i, j] = (x[i, j] - x[i-1, j])/delta # backward
                else:
                    convy[i, j] = (x[i+1, j] - x[i-1, j])/(2*delta) # central
        return convy

In [42]:
gen = Model(N, m, nb_blocks)
output = gen(x)
output_dx, output_dy = conv(output), conv(output, dx = False)
# L1, L2 = torch.pow(output_dx, 2).sum()/2, torch.pow(output_dy, 2).sum()/2
# gen_opt = torch.optim.Adam(gen.parameters(), lr=1e-4)

tensor(939.4763, grad_fn=<DivBackward0>) tensor(349.5609, grad_fn=<DivBackward0>)


In [57]:
cur_step = 0
while cur_step < 10 :
    out = gen(x)
    print(out.shape)
    cur_step += 1

torch.Size([10, 10])
torch.Size([10, 10])
torch.Size([10, 10])
torch.Size([10, 10])
torch.Size([10, 10])
torch.Size([10, 10])
torch.Size([10, 10])
torch.Size([10, 10])
torch.Size([10, 10])
torch.Size([10, 10])
